# Explore the dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

df = pd.read_csv(r'C:\Users\mohdr\Portfolio project\CSV\us_df_final_2015_2023.csv', index_col = 0)
df.set_index('date', inplace=True)
# df.info()  # Falls du Informationen über den DataFrame anzeigen möchtest
#df

### Downsample to weekly frequency by summing the daily values

In [ ]:
# Convert index to DatetimeIndex
df.index = pd.to_datetime(df.index)

# Downsample to weekly frequency by summing the daily values
df_weekly = df.resample('W').sum()
#df_weekly

# Step 1: Check for stationarity of time series
## Method #1: time series plot

In [ ]:
# Act_in_MW column is assuming
plt.figure(figsize=(10, 6))
plt.plot(df_weekly.index, df_weekly['Act_in_MW'])
plt.title('Actual_energy_erzeugung_in_MW')
plt.xlabel('date')
plt.ylabel('Value')
plt.show()

## Decomposition

### Act_in_MW' is the column want to decompose
Decomposing the Act_in_MW feature helps identify its trend, seasonal patterns, and noise. This breakdown aids in understanding its behavior over time, improving forecasting accuracy and revealing underlying patterns.

In [ ]:
decomposition = seasonal_decompose(df_weekly['Act_in_MW'], period=52)  

# Plot the decomposed components
fig = plt.figure()  
fig = decomposition.plot()  
plt.show()
#fig.savefig('fig.png')

In [ ]:
trend = decomposition.trend
seasonal = decomposition.seasonal 
residual = decomposition.resid 

# Method #2: ADF Stationarity test 
The Augmented Dickey-Fuller (ADF) test is crucial for time series analysis as it determines whether a series is stationary or not. Stationarity is a fundamental assumption for many time series models, including ARIMA. By conducting the ADF test, we can assess the stationarity of the data, which is essential for making accurate predictions and ensuring the reliability of statistical inference.

In [ ]:
from statsmodels.tsa.stattools import adfuller

def test_stationarity(timeseries):
    # Determing rolling statistics
    rolmean = timeseries.rolling(window=52).mean()
    rolstd = timeseries.rolling(window=52).std()

    # Plot rolling statistics
    fig = plt.figure(figsize=(10, 6))
    orig = plt.plot(timeseries, color='blue', label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()

    # Perform Dickey-Fuller test
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

In [ ]:
# Beispielaufruf mit der Spalte 'Act_in_MW'
test_stationarity(df_weekly['Act_in_MW'])

### The Dickey-Fuller Test resulte
The Dickey-Fuller Test resulted in a test statistic of -8.080309e+00 and a very low p-value of 1.467470e-12, indicating strong evidence against the null hypothesis of non-stationarity. Therefore, we conclude that the time series is stationary.

# Transform to stationary: differencing
Differencing may not be necessary since the data is already stationary.

### Understanding Differencing in Time Series Analysis
Differencing involves subtracting each observation from its previous observation in a time series dataset. This process helps stabilize the mean of the series by removing trends or seasonal patterns.

In [ ]:
df_weekly['first_difference'] = df['Act_in_MW'] - df_weekly['Act_in_MW'].shift(1)
test_stationarity(df_weekly['first_difference'].dropna(inplace = False))

In [ ]:
# Beispiel: Berechnung der ersten Differenz
df_weekly['first_difference'] = df_weekly['Act_in_MW'].diff()

# Beispiel: Berechnung der saisonalen Differenz
df_weekly['seasonal_first_difference'] = df_weekly['first_difference'] - df_weekly['first_difference'].shift(52)

# Test auf Stationarität
test_stationarity(df_weekly['seasonal_first_difference'].dropna(inplace=False))

# Step 2: Determine ARIMA models parameters p,d, q and seasonal_pdq
### Method #1: ACF plot and PACF plot

These plots provide insights into the autocorrelation and partial autocorrelation structures of the time series data, which are crucial for selecting the appropriate parameters for the SARIMAX model.

In [ ]:
import statsmodels.api as sm
fig = plt.figure(figsize=(8,5))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df_weekly.Act_in_MW.iloc[53:], lags=20, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df_weekly.Act_in_MW.iloc[53:], lags=20, ax=ax2)
#fig.savefig('ACF_PACF.png')

### Method #2: To determine the best parameter, cross-validation is applied
Cross-validation in time series involves iteratively partitioning data into training and validation sets to evaluate model performance while preserving the temporal order of observations.

In [ ]:
import itertools
import statsmodels.api as sm
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
import warnings

# Filter out the warning
warnings.filterwarnings("ignore")

# Parameterkombinationen für die Grid-Suche
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 52) for x in pdq]  # Wochenliche saisonale Komponente

best_aic = float("inf")
best_pdq = None
best_seasonal_pdq = None

# Initialisierung des Kreuzvalidierungssplitters
tscv = TimeSeriesSplit(n_splits=5)

# Durchführung der Kreuzvalidierung
for train_index, test_index in tscv.split(df_weekly):
    train_data = df_weekly.iloc[train_index]
    test_data = df_weekly.iloc[test_index]

    # Finden der besten Parameter mit den Trainingsdaten
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(train_data['Act_in_MW'], order=param, seasonal_order=param_seasonal, enforce_stationarity=True, enforce_invertibility=False)
                results = mod.fit()
                if results.aic < best_aic:
                    best_aic = results.aic
                    best_pdq = param
                    best_seasonal_pdq = param_seasonal
            except:
                continue


# Ausgabe der besten Parameter und des niedrigsten AIC-Werts
print('Beste Parameter (pdq, seasonal_pdq):', best_pdq, best_seasonal_pdq)
print('Bestes AIC:', best_aic)

# Step 3: Fit the SARIMAX model
Once you fit the SARIMAX model with the optimal parameters, focus on the model summary, diagnostics (like AIC and BIC), residual analysis, predictions, and forecasts. These outputs help assess the model's goodness of fit, performance, and parameter estimates.

In [ ]:
# Modell mit den besten Parametern anpassen
mod = sm.tsa.statespace.SARIMAX(df_weekly['Act_in_MW'], order=(1,1,1), seasonal_order=(1,1,0,52), enforce_stationarity=True, enforce_invertibility=False)
results = mod.fit()

# Vorhersagen auf den Daten machen
pred = results.predict()

# Fehlermetriken berechnen
mae = np.mean(np.abs(pred - df_weekly['Act_in_MW']))
mse = np.mean((pred - df_weekly['Act_in_MW'])**2)
rmse = np.sqrt(mse)

print('Mean Absolute Error (MAE):', mae)
print('Mean Squared Error (MSE):', mse)
print('Root Mean Squared Error (RMSE):', rmse)

In [ ]:
print(results.summary())

# Speichern Sie die Zusammenfassung als Bild
#plt.figure(figsize=(10, 6))
#plt.text(0.1, 0.1, results.summary().as_text(), {'fontsize': 10}, fontproperties='monospace')  # Stellen Sie die Schriftgröße und den Textstil ein
#plt.axis('off')  # Deaktivieren Sie die Achsen
#plt.savefig('summary.png', bbox_inches='tight', dpi=300)  # Speichern Sie das Bild mit hoher Qualität

#### Compare the actual values with the forecast using a graph

In [ ]:
df_weekly['forecast'] = results.predict(start = '2023-01-01', end= '2023-12-31', dynamic= False)  
df_weekly[['Act_in_MW', 'forecast']].plot(figsize=(10, 6))
plt.title('Actual vs Forecast Values', fontweight='bold', fontsize=16)
#df_weekly['forecast']
#plt.savefig('actual vs Forecast.png')

### Compare the actual values with the forecast using a graph with CI

In [ ]:
# Generiere Vorhersagen mit Konfidenzintervallen
pred = results.get_prediction(start='2023-01-01', end='2023-12-31', dynamic=False)
pred_conf = pred.conf_int()

# Plot
plt.figure(figsize=(10, 6))
plt.plot(df_weekly.index, df_weekly['Act_in_MW'], label='Actual')
plt.plot(pred.predicted_mean.index, pred.predicted_mean, color='red', label='Forecast')
plt.fill_between(pred_conf.index, pred_conf.iloc[:, 0], pred_conf.iloc[:, 1], color='pink')

plt.title('Actual vs Forecast Values with 95% Confidence Interval')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend(loc='upper left')
plt.show()
#plt.savefig('Actual vs Forecast Confidence.png')

# Step 4: Make time series predictions

### Forecast future values (e.g., for next 52 steps)

In [ ]:
forecast_steps = 52
# Get forecast object
forecast = results.get_forecast(steps=forecast_steps)

# Extract forecasted values
forecast_values = forecast.predicted_mean

# Extract standard errors of forecasted values
forecast_errors = forecast.se_mean

# Define confidence levels
alpha_95 = 0.05
alpha_90 = 0.1

# Calculate confidence intervals based on forecast errors
conf_int_95 = [(val - 1.96 * err, val + 1.96 * err) for val, err in zip(forecast_values, forecast_errors)]
conf_int_90 = [(val - 1.645 * err, val + 1.645 * err) for val, err in zip(forecast_values, forecast_errors)]

# Visualization
plt.figure(figsize=(10, 6))
plt.plot(df_weekly.index, df_weekly.Act_in_MW, label='Actual')
plt.plot(forecast_values.index, forecast_values, label='Forecast', color='red')
plt.fill_between(forecast_values.index, [x[0] for x in conf_int_95], [x[1] for x in conf_int_95], color='pink', alpha=0.8, label='95% CI')
#plt.fill_between(forecast_values.index, [x[0] for x in conf_int_90], [x[1] for x in conf_int_90], color='orange', alpha=0.3, label='90% CI')
plt.title('Actual vs Predicted Values with Confidence Intervals', fontweight='bold', fontsize=16)
plt.xlabel('Time')
plt.ylabel('Values')
plt.legend(loc='upper left')
#plt.savefig('actualforcasteCI.png')
plt.show()

# Step 5: Evaluate model predictions

### Calculate residuals

In [ ]:
import seaborn as sns
import statsmodels.api as sm

# Berechnen Sie die Residuen
residuals = results.resid

# Plot der Residuen ab 2016
plt.figure(figsize=(12, 6))

# Plot der Residuen
plt.subplot(1, 2, 1)
plt.plot(residuals)
plt.title('Residuals')
plt.xlabel('date')
plt.ylabel('Residuen')

# Dichtediagramm
plt.subplot(1, 2, 2)
residuals.plot(kind='kde')

plt.tight_layout()
plt.savefig('residuals.png')
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition_re = seasonal_decompose(residuals)  

# Plot the decomposed components
fig = plt.figure()  
fig = decomposition_re.plot()  
plt.show()


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Plot der Autokorrelationsfunktion (ACF)
plt.figure(figsize=(12, 4))
plot_acf(residuals, lags=40, ax=plt.gca())
plt.title('Autokorrelationsfunktion (ACF) der Residuen')
plt.xlabel('Lag')
plt.ylabel('Autokorrelation')
plt.show()

# Plot der partiellen Autokorrelationsfunktion (PACF)
plt.figure(figsize=(12, 4))
plot_pacf(residuals, lags=40, ax=plt.gca())
plt.title('Partielle Autokorrelationsfunktion (PACF) der Residuen')
plt.xlabel('Lag')
plt.ylabel('Partielle Autokorrelation')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import probplot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Berechnen der Residuen
residuals = results.resid

# Erstellen des Gitters
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

# Normal probability plot of residuals (Normalwahrscheinlichkeitsplot der Residuen)
probplot(residuals, plot=axs[0, 0])
axs[0, 0].set_title('Normal Probability Plot of Residuals', fontweight='bold', fontsize=16)
axs[0, 0].set_xlabel('Theoretical Quantiles')
axs[0, 0].set_ylabel('Sample Quantiles')

# Residuals versus fitted values (Residuen versus angepasste Werte)
pred = results.fittedvalues  # Assuming 'results' is the variable storing the model results
axs[0, 1].scatter(pred, residuals, alpha=0.5)
axs[0, 1].axhline(y=0, color='red', linestyle='--')
axs[0, 1].set_title('Residuals versus Fitted Values', fontweight='bold', fontsize=16)
axs[0, 1].set_xlabel('Fitted Values')
axs[0, 1].set_ylabel('Residuals')

# ACF/PACF plot of residuals (ACF/PACF-Plot der Residuen)
plot_acf(residuals, lags=40, ax=axs[1, 0])
plot_pacf(residuals, lags=40, ax=axs[1, 1])
axs[1, 0].set_title('ACF Plot of Residuals', fontweight='bold', fontsize=16)
axs[1, 1].set_title('PACF Plot of Residuals', fontweight='bold', fontsize=16)

# Layout anpassen
plt.tight_layout()

# Speichern des Plots
plt.savefig('combined_plots.png')
plt.show()

# Shapiro-Wilk-Test auf Normalverteilung der Residuen

In [ ]:
from scipy import stats

# Shapiro-Wilk-Test auf Normalverteilung der Residuen
shapiro_test_stat, shapiro_p_value = stats.shapiro(residuals)
print('Shapiro-Wilk-Test Statistik:', shapiro_test_stat)
print('Shapiro-Wilk-Test p-Wert:', shapiro_p_value)

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Visual Diagnostics
# Plot the residuals
plt.figure(figsize=(10, 6))
plt.plot(residuals)
plt.title('Residuals over Time', fontweight='bold', fontsize=16)
plt.xlabel('Time')
plt.ylabel('Residuals')
#plt.savefig('Residuals over Time.png')
plt.show()
